In [49]:
import os
import pandas as pd

directory = os.chdir("G:\My Drive\Capstone Files\Datasets")

<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\charlesrowe\AppData\Local\Temp\ipykernel_21464\1295670377.py:4: SyntaxWarning: invalid escape sequence '\M'
  directory = os.chdir("G:\My Drive\Capstone Files\Datasets")


In [50]:
df = pd.read_csv("cleaned_full_df_with_census_regions.csv")

cols_to_drop = df[(df['census region'] == 'northeast') | (df['census region'] == 'midwest')].index
df.drop(cols_to_drop, inplace=True)

import demoji
import re

def remove_emoji(string):
    return demoji.replace(string, '')

def remove_extra_spaces(string):
    return re.sub(r'\s{2,}',' ',str(string))

df['cleaned text'] = df['Review Text'].apply(remove_emoji)
df['cleaned text'] = df['cleaned text'].apply(remove_extra_spaces)

import nltk
import string
import unicodedata
import nltk.corpus
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_sentence(sentence):
    stops = nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...',' - ', 'ca', 'wo', "'s", "'ing","'ll", "'re"]
    negation = ['but', 'not', "don't", "aren't", "couldn't", "doesn't", "hadn't", "hasn't", "haven't", 
                  "isn't", "shouldn't", "wouldn't"]
    for word in negation:
        stops.remove(word)
    sentence = (unicodedata.normalize('NFKD', sentence)
        .encode('ascii', 'ignore')
        .decode('utf-8', 'ignore')
        .lower())
    words = re.sub(r'[^\w\s]', '', sentence).split()
    word_list = [word for word in words if word not in stops]
    return word_list

def get_words(df, column):
    return clean_sentence(''.join(str(df[column].tolist())))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charlesrowe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\charlesrowe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charlesrowe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\charlesrowe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
mask = df['census region'] == 'southern'
south_df = df[mask]
west_df = df[~mask]

south_df['cleaned text'] = south_df['cleaned text'].str.lower()

C:\Users\charlesrowe\AppData\Local\Temp\ipykernel_21464\3838607419.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  south_df['cleaned text'] = south_df['cleaned text'].str.lower()


In [ ]:
def context(pattern, dataframe):
    start_context = pd.Series(dataframe['cleaned text'])
    dataframe['context'] = [re.search(pattern,i) for i in start_context]
    context_df = dataframe.dropna(subset=['context'])
    return context_df['context']

context(r'(\w+\s+){0,3}clearly(\s+\S+){0,3}',south_df)

C:\Users\charlesrowe\AppData\Local\Temp\ipykernel_21464\2853981048.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['context'] = [re.search(pattern,i) for i in start_context]


11        <re.Match object; span=(229, 255), match='clea...
94        <re.Match object; span=(53, 80), match='clearl...
190       <re.Match object; span=(363, 395), match='clea...
250       <re.Match object; span=(135, 142), match='clea...
261       <re.Match object; span=(50, 70), match='clearl...
                                ...                        
198134    <re.Match object; span=(0, 19), match='clearly...
198237    <re.Match object; span=(242, 266), match='clea...
198440    <re.Match object; span=(267, 295), match='clea...
198472    <re.Match object; span=(549, 570), match='clea...
198531    <re.Match object; span=(569, 589), match='clea...
Name: context, Length: 940, dtype: object

In [61]:
text = "love top you for why is this clearly not working correctly and I can't work"

re.search(r'clearly(\s+\S+){0,3}',text)

<re.Match object; span=(29, 58), match='clearly not working correctly'>